In [1]:
import torch
import torch.nn as nn
import numpy as np

# Master branch

In [2]:
class MasterNet(nn.Module):
    def __init__(self):
        super(MasterNet, self).__init__()
        
    def forward(self,x):
        return x

# Acoustic Branch

Inputs for acoustic branch will be N x 40 where N [1,33]  
Time step: (2, 10) (seconds?)  
N: relative duration after feature extraction

In [17]:
class AcousticNet(nn.Module):
    def __init__(self, num_conv_layers = 3, num_gru_layers = 2, kernel_size = 2, conv_width = 32):
        super(AcousticNet, self).__init__()
        self.num_conv_layers = num_conv_layers
        self.conv1 = nn.Conv1d(in_channels=40, out_channels=conv_width, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(in_channels=conv_width, out_channels=conv_width, kernel_size=kernel_size)
        self.conv3 = nn.Conv1d(in_channels=conv_width, out_channels=conv_width, kernel_size=kernel_size)
        self.conv4 = nn.Conv1d(in_channels=conv_width, out_channels=conv_width, kernel_size=kernel_size)
        self.convs = [self.conv1, self.conv2, self.conv3, self.conv4]
        self.max_pool = nn.MaxPool1d(kernel_size = 2)
        self.relu = nn.ReLU()
        
        self.gru = nn.GRU(input_size=conv_width,hidden_size=32,num_layers=num_gru_layers) # 19 is hardcoded
        self.mean_pool = nn.AvgPool1d(kernel_size=2)
        
    def forward(self, x):
        print("initial x shape",x.shape)
        for i in range(self.num_conv_layers):
            x = self.relu(self.max_pool(self.convs[i](x)))
        print('after final conv', x.shape)
        x = torch.transpose(x, 1, 2)
        print('after transpose', x.shape)
        x, _ = self.gru(x)
        print('gru', x.shape)
        x = self.mean_pool(x)
        print('mean pool', x.shape)
        return x

In [20]:
# Test dummy input
net = AcousticNet(num_conv_layers = 3, num_gru_layers = 2, kernel_size = 3, conv_width = 32)
test_vec = torch.randn(10, 40, 17) # samples x features (or channels) x N (relative duration)
output = net(test_vec)
print(f'Shape of output: {output.shape}')
# assert output.shape[-1] == 16

initial x shape torch.Size([10, 40, 17])


RuntimeError: Calculated padded input size per channel: (2). Kernel size: (3). Kernel size can't be greater than actual input size

# Lexical Branch

In [5]:
# implement GRU (or transformer)
class LexicalNet(nn.Module):
    def __init__(self, num_gru_layers = 2):
        super(LexicalNet, self).__init__()
        # implement GRU (or transformer)
        self.gru = nn.GRU(input_size=300,hidden_size=32,num_layers=num_gru_layers)
        self.mean_pool = nn.AvgPool1d(kernel_size=2) 
        
    def forward(self, x):
        x, _ = self.gru(x)
        x = self.mean_pool(x)
        return x

In [6]:
# Test dummy input
net = LexicalNet(num_gru_layers = 3)
test_vec = torch.randn(10, 1, 300)
output = net(test_vec)
assert output.shape[-1] == 16